# Web scraping Stack Overflow

We import the libraries we are going to use.

In [1]:
import os
import time
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

import json
from lxml import html
import re
import csv
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    WebDriverException,
    )
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def get_SOS_help(command):
    
    try:
        exec(command)
        
    except Exception as e:
        #Save the error message
        error_tipo = None 
        error_mensaje = None
        error_tipo = type(e).__name__
        error_mensaje = str(e)
        error_tot = error_tipo + ': ' + error_mensaje

        #Open the Chrome browser
        chrome_options = Options()
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.wait = WebDriverWait(driver, 20)
        
        #We go to the Stack Overflow website
        url = "https://stackoverflow.com/"
        driver.get(url)
        
        #Consent Cookies
        button= WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.ID, 'onetrust-accept-btn-handler')))
        button.click()
        
        #We enter the error message in the stackoverflow search bar
        search_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".s-topbar .s-topbar--searchbar .s-topbar--searchbar--input-group .s-input"))
        )
        search_input.send_keys(error_tot)
        search_input.submit()
        
        #Wait for the page to load after searching
        WebDriverWait(driver, 10).until(EC.title_contains("Stack Overflow"))

        #We introduce an infinite loop until the Captcha is completed, once done the code will continue.
        while True:
            try:
                #We filter by the most voted
                desplegar = driver.find_element(By.CSS_SELECTOR, "nav.d-flex.ps-relative")
                desplegar.click()
                masvoto1 = driver.find_element(By.CSS_SELECTOR, '[title="Highest scored search results"]')
                masvoto1.click()
        
                try:
                    #Select the answer with the most votes      
                    elemento = driver.find_element(By.CLASS_NAME, 's-post-summary--content-title')
                    best_ans = elemento.find_element(By.CLASS_NAME, 's-link')
                    best_ans.click()
            
                    #We print the text of the most voted answer and the URL of where it is located.
                    answercell_elemento = driver.find_element(By.CLASS_NAME, 'answercell.post-layout--right')
                    s_prose_elemento = answercell_elemento.find_element(By.CLASS_NAME, 's-prose.js-post-body')
                    print('Se ha producido el siguiente error al intentar ejecutar el código: ', error_tot)
                    print('La solución que te ofrece la web de Stack Overflow es:')
                    print('\n', s_prose_elemento.text)
            
                    current_url = driver.current_url
                    print('\nLa fuente de donde sacamos esta respuesta es: ', current_url)

                    

                #In case there are no answers for the given error, print that no answers have been found.
                except:
                    print('No se ha encontrado una solución para el error producido: ', error_tot)
                
                driver.quit()    
                break

            #With the following exception we keep the loop running until the Captcha is completed.
            except:
                continue
           
    return

## Create a malfunctioning code and use this function on it

In [3]:
command = 'nn.random.uniform(-1, 1, size=100)'
get_SOS_help(command)

Se ha producido el siguiente error al intentar ejecutar el código:  NameError: name 'nn' is not defined
La solución que te ofrece la web de Stack Overflow es:

 If you got this error you can fix it with the following code:
import torch
import torch.nn as nn
You need to include both lines, since if you set just the second one it may not work if the torch package is not imported.
Where torch and torch.nn (or just nn) are two of the main PyTorch packages. You can help(torch.nn) to confirm this.
It is not uncommon when you include nn to include the functional interface as F like this:
import torch
import torch.nn as nn
import torch.nn.functional as F
To bring you the hints what you imported or what is inside the nn package I provided the list:
['AdaptiveAvgPool1d', 'AdaptiveAvgPool2d', 'AdaptiveAvgPool3d', 'AdaptiveLogSoftmaxWithLoss', 'AdaptiveMaxPool1d', 'AdaptiveMaxPool2d', 'AdaptiveMaxPool3d', 'AlphaDropout', 'AvgPool1d', 'AvgPool2d', 'AvgPool3d', 'BCELoss', 'BCEWithLogitsLoss', 'Batch